# 爬虫之IP代理，浏览器自动化及模拟登陆

在利用爬虫进行访问网页时，经常会遇到网页限制的情况，这常常是由于网站对同一IP在短时间进行大量重复访问时采取的限制措施，如何解决这种方法呢，这里主要有三种方法：
~~虽然我觉得这样不太好，本来对网页的重复访问，对服务器就添加了很多额外的压力，所以服务器对你进行限制也是正常的，为了摆脱这种限制，使用额外的技术手段，还是达到了同样的目的，不能不说服务器太难了。就不能换一个方式和平解决嘛哈哈，比如约定一个时间段，比如凌晨3点到6点，这时候服务器压力小，允许你爬虫，当然咱们为了取得一些资料有时候还是不得不这样做~~

1. 模拟浏览器访问
2. 在每次访问时添上一个时间间隔，不要那么频繁
3. 使用IP代理

#### 1.模拟浏览器访问

对于第一个方法，就是加上表头，及浏览器的身份信息，最好还需要加上时间间隔，不然对浏览器大量重复访问服务器也是会限制的。

In [4]:
from bs4 import BeautifulSoup
import requests
import time

def open_proxy_url(url):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

    try:
        r = requests.get(url, headers = headers, timeout = 20)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return(r.text)
    except:
        print('无法访问网页' + url)

def get_proxy_ip(response):
    proxy_ip_list = []
    soup = BeautifulSoup(response, 'html.parser')
    proxy_ips = soup.find(id = 'ip_list').find_all('tr')
    for proxy_ip in proxy_ips:
        if len(proxy_ip.select('td')) >=8:
            ip = proxy_ip.select('td')[1].text
            port = proxy_ip.select('td')[2].text
            protocol = proxy_ip.select('td')[5].text
            if protocol in ('HTTP','HTTPS','http','https'):
                proxy_ip_list.append(f'{protocol}://{ip}:{port}')
    return proxy_ip_list

if __name__ == '__main__':
    proxy_url = 'https://www.xicidaili.com/'
    text = open_proxy_url(proxy_url)
    proxy_ip_filename = 'proxy_ip.txt'
    with open(proxy_ip_filename, 'w') as f:
        f.write(text)
    text = open(proxy_ip_filename, 'r').read()
    proxy_ip_list = get_proxy_ip(text)
    print(proxy_ip_list)

['HTTPS://117.45.139.84:9006', 'HTTP://114.223.208.165:8118', 'HTTPS://171.35.86.72:8118', 'HTTP://49.235.246.24:8118', 'HTTPS://114.223.103.47:8118', 'HTTP://58.215.201.98:35728', 'HTTP://60.188.65.73:3000', 'HTTP://49.235.69.138:8118', 'HTTP://112.14.47.6:52024', 'HTTP://101.132.190.101:80', 'HTTP://222.85.28.130:40505', 'HTTP://175.148.69.26:1133', 'HTTPS://183.166.103.164:9999', 'HTTP://113.195.225.68:9999', 'HTTP://106.14.173.173:8080', 'HTTPS://117.88.176.123:3000', 'HTTP://175.148.68.175:1133', 'HTTPS://171.35.163.132:9999', 'HTTPS://60.31.213.115:808', 'HTTP://117.88.177.24:3000', 'HTTPS://113.78.255.93:9000', 'HTTPS://60.191.11.237:3128', 'HTTPS://58.255.38.156:9000', 'HTTP://14.115.107.232:808', 'HTTP://163.125.71.195:8888', 'HTTP://123.163.24.113:3128', 'HTTP://222.240.184.126:8086', 'HTTPS://117.141.155.241:53281', 'HTTPS://218.22.7.62:53281', 'HTTP://163.125.71.198:9999', 'HTTPS://14.153.52.10:3128', 'HTTPS://61.164.39.69:53281', 'HTTP://27.38.155.190:8118', 'HTTPS://113.6

In [5]:
from bs4 import BeautifulSoup
import requests
import re
import json


def open_proxy_url(url):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

    try:
        r = requests.get(url, headers = headers, timeout = 10)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        print('无法访问网页' + url)

def get_proxy_ip(response):
    proxy_ip_list = []
    soup = BeautifulSoup(response, 'html.parser')
    proxy_ips = soup.find(id = 'ip_list').find_all('tr')
    for proxy_ip in proxy_ips:
        if len(proxy_ip.select('td')) >=8:
            ip = proxy_ip.select('td')[1].text
            port = proxy_ip.select('td')[2].text
            protocol = proxy_ip.select('td')[5].text
            if protocol in ('HTTP','HTTPS','http','https'):
                proxy_ip_list.append(f'{protocol}://{ip}:{port}')
    return proxy_ip_list


def open_url_using_proxy(url, proxy):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

    proxies = {}
    if proxy.startswith(('HTTPS','https')):
        proxies['https'] = proxy
    else:
        proxies['http'] = proxy

    try:
        r = requests.get(url, headers = headers, proxies = proxies, timeout = 10)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return (r.text, r.status_code)
    except:
        print('无法访问网页' + url)
        print('无效代理IP: ' + proxy)
        return False

def check_proxy_avaliability(proxy):
    url = 'http://www.baidu.com'
    result = open_url_using_proxy(url, proxy)
    VALID_PROXY = False
    if result:
        text, status_code = result
        if status_code == 200:
            r_title = re.findall('<title>.*</title>', text)
            if r_title:
                if r_title[0] == '<title>百度一下，你就知道</title>':
                    VALID_PROXY = True
        if VALID_PROXY:
            check_ip_url = 'https://jsonip.com/'
            try:
                text, status_code = open_url_using_proxy(check_ip_url, proxy)
            except:
                return

            print('有效代理IP: ' + proxy)
            with open('valid_proxy_ip.txt','a') as f:
                f.writelines(proxy)
            try:
                source_ip = json.loads(text).get('ip')
                print(f'源IP地址为：{source_ip}')
                print('='*40)
            except:
                print('返回的非json,无法解析')
                print(text)
    else:
        print('无效代理IP: ' + proxy)
        
if __name__ == '__main__':
    proxy_url = 'https://www.xicidaili.com/'
    proxy_ip_filename = 'proxy_ip.txt'
    text = open(proxy_ip_filename, 'r').read()
    proxy_ip_list = get_proxy_ip(text)
    for proxy in proxy_ip_list:
        check_proxy_avaliability(proxy)

无法访问网页http://www.baidu.com
无效代理IP: HTTPS://117.45.139.84:9006
无效代理IP: HTTPS://117.45.139.84:9006
无法访问网页http://www.baidu.com
无效代理IP: HTTP://114.223.208.165:8118
无效代理IP: HTTP://114.223.208.165:8118
无法访问网页http://www.baidu.com
无效代理IP: HTTPS://171.35.86.72:8118
无效代理IP: HTTPS://171.35.86.72:8118
无法访问网页http://www.baidu.com
无效代理IP: HTTP://49.235.246.24:8118
无效代理IP: HTTP://49.235.246.24:8118
无法访问网页http://www.baidu.com
无效代理IP: HTTPS://114.223.103.47:8118
无效代理IP: HTTPS://114.223.103.47:8118
无法访问网页http://www.baidu.com
无效代理IP: HTTP://58.215.201.98:35728
无效代理IP: HTTP://58.215.201.98:35728
无法访问网页http://www.baidu.com
无效代理IP: HTTP://60.188.65.73:3000
无效代理IP: HTTP://60.188.65.73:3000
无法访问网页http://www.baidu.com
无效代理IP: HTTP://49.235.69.138:8118
无效代理IP: HTTP://49.235.69.138:8118
无法访问网页http://www.baidu.com
无效代理IP: HTTP://112.14.47.6:52024
无效代理IP: HTTP://112.14.47.6:52024
有效代理IP: HTTP://101.132.190.101:80
源IP地址为：222.70.252.35
无法访问网页http://www.baidu.com
无效代理IP: HTTP://222.85.28.130:40505
无效代理IP: HTTP://222.8

### 2.selenium

selenium是一个自动化测试工具,可以用代码的方式去模拟浏览器操作过程（如：打开浏览器、在输入框里输入文字、回车等），在使用前需要先安装selenium（pip install selenium）并且下载安装chromedriver（一个驱动程序，用以启动chrome浏览器，具体的驱动程序需要对应的驱动，在官网上可以找到下载地址） 

In [6]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys  # 提供键盘按键支持

In [15]:
driver = webdriver.Chrome(r"E:\Machine Learning\PaChong\task0\chromedriver.exe")

In [16]:
driver.get("https://www.jianshu.com/p/191d1e21f7ed/")  # 这个时候chromedriver会打开一个Chrome浏览器窗口，显示的是网址所对应的页面

In [13]:
driver.get("https://www.baidu.com") 

In [14]:
driver.close()  # 关闭浏览器一个Tab
# or
#driver.quit()  # 关闭浏览器窗口

In [20]:
element = driver.find_element_by_name("q")

In [21]:
element.send_keys(“some text”）  # 往一个可以输入对象中输入“some text”
#甚至

element.send_keys(Keys.RETURN）  # 模拟键盘回车
#一般来说，这种方式输入后会一直存在，而要清空某个文本框中的文字，就需要：

#element.clear()  # 清空element对象中的文字

SyntaxError: invalid character in identifier (<ipython-input-21-adaf01ba3cc5>, line 1)

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver = webdriver.Firefox()
driver.get("http://somedomain/url_that_delays_loading")
try:    
    element = WebDriverWait(driver, 10).until(           
        EC.presence_of_element_located((By.ID, "myDynamicElement")))
finally:    
    driver.quit()
#其中，presence_of_element_located是条件，By.ID是通过什么方式来确认元素（这个是通过id），"myDynamicElement"这个就是某个元素的ID

In [ ]:
driver = webdriver.Firefox()
driver.implicitly_wait(10) # seconds
driver.get("http://somedomain/url_that_delays_loading")
myDynamicElement = driver.find_element_by_id("myDynami")

### 3. session和cookie

session 是服务器端的会话，但我不知道要具体怎么去描述它，就是服务器端的一个进程吧。cookie是你登录后，服务器发给你的一个标记，以验证你的身份的。这样，根据cookie，服务器知道你是谁，浏览过哪些东西，可以个性化的给你推荐东西。这个就是动态网页的好处。静态网页就比较简单，所有东西都是事先写好的，你去访问时，拿出来给你看就行。下面是资料解释。

动态网页和静态网页

静态网页

静态网页就是我们上一篇写的那种 html 页面，后缀为 .html 的这种文件，直接部署到或者是放到某个 web 容器上，就可以在浏览器通过链接直接访问到了，常用的 web 容器有 Nginx 、 Apache 、 Tomcat 、Weblogic 、 Jboss 、 Resin 等等，很多很多。举个例子：https://desmonday.github.io/，就是静态网页的代表，这种网页的内容是通过纯粹的 HTML 代码来书写，包括一些资源文件：图片、视频等内容的引入都是使用 HTML 标签来完成的。它的好处当然是加载速度快，编写简单，访问的时候对 web 容器基本上不会产生什么压力。但是缺点也很明显，可维护性比较差，不能根据参数动态的显示内容等等。有需求就会有发展么，这时动态网页就应运而生了

动态网页

大家常用的某宝、某东、拼夕夕等网站都是由动态网页组成的。
动态网页可以解析 URL 中的参数，或者是关联数据库中的数据，显示不同的网页内容。现在各位同学访问的网站大多数都是动态网站，它们不再简简单单是由 HTML 堆砌而成，可能是由 JSP 、 PHP 等语言编写的，当然，现在很多由前端框架编写而成的网页小编这里也归属为动态网页。



In [ ]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:

"""
使用selenium进行模拟登陆
1.初始化ChromDriver
2.打开163登陆页面
3.找到用户名的输入框，输入用户名
4.找到密码框，输入密码
5.提交用户信息
"""
name = '*'
passwd = '*'
driver = webdriver.Chrome('./chromedriver')
driver.get('https://mail.163.com/')
# 将窗口调整最大
driver.maximize_window()
# 休息5s
time.sleep(5)
current_window_1 = driver.current_window_handle
print(current_window_1)

In [ ]:
button = driver.find_element_by_id('lbNormal')
button.click()

driver.switch_to.frame(driver.find_element_by_xpath("//iframe[starts-with(@id, 'x-URS-iframe')]"))

In [ ]:
email = driver.find_element_by_name('email')
#email = driver.find_element_by_xpath('//input[@name="email"]')
email.send_keys(name)
password = driver.find_element_by_name('password')
#password = driver.find_element_by_xpath("//input[@name='password']")
password.send_keys(passwd)
submit = driver.find_element_by_id("dologin")
time.sleep(15)
submit.click()
time.sleep(10)
print(driver.page_source)
driver.quit()

今天做模型，实在太累了，就不写小结了。这一期就水过去啦，助教原谅我~